# Step-1: Loading the Dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import glob 
filePaths = glob.glob("/home/prakhar/Desktop/College/3rdYear/SEM-VI/MinorProject/Datasets/archive-2/*.csv")
df_list = [pd.read_csv(file,low_memory=False) for file in filePaths]
df=pd.concat(df_list,ignore_index=True)
print(df.shape)
print(df.head())

(2830743, 79)
    Destination Port   Flow Duration   Total Fwd Packets  \
0              54865               3                   2   
1              55054             109                   1   
2              55055              52                   1   
3              46236              34                   1   
4              54863               3                   2   

    Total Backward Packets  Total Length of Fwd Packets  \
0                        0                           12   
1                        1                            6   
2                        1                            6   
3                        1                            6   
4                        0                           12   

    Total Length of Bwd Packets   Fwd Packet Length Max  \
0                             0                       6   
1                             6                       6   
2                             6                       6   
3                             6   

In [3]:
df.columns = df.columns.str.strip()
df['Label'].unique()

array(['BENIGN', 'DDoS', 'PortScan', 'Bot', 'Infiltration',
       'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection', 'FTP-Patator', 'SSH-Patator',
       'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye',
       'Heartbleed'], dtype=object)

# Step-2: Pre-Processing

In [4]:
df.dtypes #Viewing Datatype

Destination Port                 int64
Flow Duration                    int64
Total Fwd Packets                int64
Total Backward Packets           int64
Total Length of Fwd Packets      int64
                                ...   
Idle Mean                      float64
Idle Std                       float64
Idle Max                         int64
Idle Min                         int64
Label                           object
Length: 79, dtype: object

In [5]:
is_inf = df.isin([np.inf, -np.inf])

inf_counts = is_inf.sum()

# Filter to show only columns with infinite values
columns_with_inf = inf_counts[inf_counts > 0]
print("Columns with infinite values:\n", columns_with_inf)

nan_counts = df.isnull().sum()

# Filter to show only columns with NaN values
columns_with_nan = nan_counts[nan_counts > 0]
print("\nColumns with NaN values:\n", columns_with_nan)

combined_counts = nan_counts + inf_counts

# Filter to show only columns with any NaN or inf values
columns_with_nan_or_inf = combined_counts[combined_counts > 0]
print("\nColumns with NaN or infinite values:\n", columns_with_nan_or_inf)

duplicate_rows_count = df.duplicated().sum()
print("\nNumber of duplicate rows:", duplicate_rows_count)

Columns with infinite values:
 Flow Bytes/s      1509
Flow Packets/s    2867
dtype: int64

Columns with NaN values:
 Flow Bytes/s    1358
dtype: int64

Columns with NaN or infinite values:
 Flow Bytes/s      2867
Flow Packets/s    2867
dtype: int64

Number of duplicate rows: 308381


In [6]:
df.replace([np.inf, -np.inf], np.nan, inplace=True) #Replacing inf values
df.dropna(inplace=True) #Replacing NaN values
df.drop_duplicates(inplace=True) #Removing duplicates to improve accuracy
category_mapping = {
    'BENIGN': 'BENIGN',
    'Bot': 'BOTNET',
    'DDoS': 'DOS',
    'DoS GoldenEye': 'DOS',
    'DoS Hulk': 'DOS',
    'DoS Slowhttptest': 'DOS',
    'DoS slowloris': 'DOS',
    'FTP-Patator': 'BRUTE_FORCE',
    'SSH-Patator': 'BRUTE_FORCE',
    'Heartbleed': 'WEB_ATTACK',
    'Infiltration': 'WEB_ATTACK',
    'PortScan': 'RECONNAISSANCE',
    'Web Attack � Brute Force': 'WEB_ATTACK',
    'Web Attack � Sql Injection': 'WEB_ATTACK',
    'Web Attack � XSS': 'WEB_ATTACK'
}
#Applying Mapping to simplify labels
df['Label'] =df['Label'].map(category_mapping)

In [7]:
print(df['Label'].value_counts())  # Before encoding

Label
BENIGN            2095057
DOS                321759
RECONNAISSANCE      90694
BRUTE_FORCE          9150
WEB_ATTACK           2190
BOTNET               1948
Name: count, dtype: int64


In [8]:
from sklearn.preprocessing import LabelEncoder #After Encoding
le = LabelEncoder()
df['Label'] = le.fit_transform(df['Label'])

In [9]:
print(df['Label'].value_counts())  # Before encoding

Label
0    2095057
3     321759
4      90694
2       9150
5       2190
1       1948
Name: count, dtype: int64


In [10]:
import pandas as pd

# Get class counts
class_counts = df['Label'].value_counts()

# Select classes with more than 5000 samples
selected_classes = class_counts[class_counts > 50000].index.tolist()

# Manually include "WEB_ATTACK" and "BOTNET" if they exist in the dataset
for attack in [1, 5, 2]:
    if attack in df['Label'].values and attack not in selected_classes:
        selected_classes.append(attack)

# Select rows where 'Label' is in selected_classes
selected = df[df['Label'].isin(selected_classes)]

dfs = []
for name in selected_classes:
    df_class = selected[selected['Label'] == name]

    # Downsample if the class has more than 5000 samples
    if len(df_class) > 50000:
        df_class = df_class.sample(n=50000, random_state=0)

    dfs.append(df_class)

# Combine all sampled data
balanced_df = pd.concat(dfs, ignore_index=True)

# Check final class distribution
print(balanced_df['Label'].value_counts())

Label
0    50000
3    50000
4    50000
2     9150
5     2190
1     1948
Name: count, dtype: int64


In [11]:
balanced_df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,53,62874,4,2,116,186,29,29,29.000,0.000000,...,20,0.00000,0.00000,0,0,0.000,0.00000,0,0,0
1,57225,86,1,1,0,0,0,0,0.000,0.000000,...,32,0.00000,0.00000,0,0,0.000,0.00000,0,0,0
2,80,116003861,16,14,440,412,356,0,27.500,87.612784,...,20,46441.09091,31255.09835,140678,36852,9995557.727,14892.00092,10000000,9950679,0
3,443,5683083,8,6,397,3716,216,0,49.625,79.224703,...,20,269399.00000,0.00000,269399,269399,5413681.000,0.00000,5413681,5413681,0
4,53,23652,1,1,48,128,48,48,48.000,0.000000,...,20,0.00000,0.00000,0,0,0.000,0.00000,0,0,0


In [12]:
from imblearn.over_sampling import SMOTE

x = balanced_df.drop('Label', axis=1)
y = balanced_df['Label']

smote = SMOTE(sampling_strategy='auto', random_state=0)
X_upsampled, y_upsampled = smote.fit_resample(x, y)

blnc_data = pd.DataFrame(X_upsampled)
blnc_data['Label'] = y_upsampled
blnc_data = blnc_data.sample(frac=1)

print(blnc_data['Label'].value_counts())
blnc_data.head()

Label
2    50000
4    50000
3    50000
0    50000
1    50000
5    50000
Name: count, dtype: int64


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
226743,21,239,2,1,14,0,14,0,7.000000,9.899495,...,32,0.0,0.0,0,0,0.0,0.0,0,0,2
212828,22,11365057,21,32,2008,2745,640,0,95.619048,140.045163,...,32,0.0,0.0,0,0,0.0,0.0,0,0,2
114704,2200,28,1,1,2,6,2,2,2.000000,0.000000,...,24,0.0,0.0,0,0,0.0,0.0,0,0,4
99484,80,85542529,7,7,728,11595,364,0,104.000000,177.613813,...,32,1.0,0.0,1,1,85300000.0,0.0,85300000,85300000,3
12970,53,31778,2,2,88,246,44,44,44.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0


In [13]:
from sklearn.decomposition import IncrementalPCA
from sklearn.preprocessing import StandardScaler
x = blnc_data.drop('Label',axis=1)
y = blnc_data['Label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(x)
n_components = 20
ipca = IncrementalPCA(n_components=n_components, batch_size=500)
x_pca = ipca.fit_transform(X_scaled) 
print(f"Explained Variance Ratio: {sum(ipca.explained_variance_ratio_):.4f}")
x = x_pca

Explained Variance Ratio: 0.9314


In [14]:
pca_df = pd.DataFrame(x, columns=[f'PC{i+1}' for i in range(x.shape[1])])
print(pca_df)

             PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
0      -1.642330 -0.183233 -1.090585  0.473777 -0.636947 -3.137814  0.991939   
1      -1.080084  0.350861  0.602672 -0.462256  0.679218  1.157663 -0.220273   
2      -1.889934 -0.068200 -0.021704  0.060919 -0.084264  0.157771 -0.157232   
3       9.231434 -0.929868 -0.009307  0.898754 -1.221010 -1.137010  0.224585   
4      -1.617491 -0.111465 -0.268196  0.025182 -0.046396 -1.245170 -1.213853   
...          ...       ...       ...       ...       ...       ...       ...   
299995 -1.504319 -0.127403 -0.490868 -0.135488  0.235828  1.937032  0.156945   
299996 -1.675080  0.038403  0.222695 -0.145582  0.214967  0.634224  0.029415   
299997 -1.517505  0.055530 -0.148289 -0.201020  0.310720  1.368891 -0.159123   
299998 -1.802997 -0.054102 -0.086467 -0.042035  0.074915  0.646962 -0.220230   
299999 -1.897820 -0.068520 -0.018651  0.053607 -0.073273  0.141588 -0.166357   

             PC8       PC9      PC10   

In [15]:
pca_df.shape

(300000, 20)

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,shuffle=True)

# Step-3: Applying Classifier Algorithms

In [18]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import RocCurveDisplay
import tensorflow as tf
import keras
from keras import layers,regularizers,optimizers

2025-03-18 17:27:32.318464: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-18 17:27:32.322285: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-18 17:27:32.330669: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742299052.344115   38101 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742299052.347793   38101 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-18 17:27:32.361994: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

#### Step-3.1: Applying Random Foressts

In [ ]:
from sklearn.ensemble import RandomForestClassifier #Importing Random Forest classifier
rf = RandomForestClassifier(n_estimators=1,max_depth=7,random_state=42)
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test) #Performing Random Forest

In [ ]:
print(classification_report(y_test, y_pred)) #Printing Classification Report
class_indices = rf.classes_
print(class_indices)

In [ ]:
cv_rf = cross_val_score(rf, x_train, y_train, cv=5)
print(f'\nCross-validation scores:', ', '.join(map(str, cv_rf)))
print(f'\nMean cross-validation score: {cv_rf.mean():.2f}')

In [ ]:
print(rf.score(x_train,y_train)) #RF Score x_train,y_train

In [ ]:
print(rf.score(x_test, y_test)) #RF Score x_test,y_test

In [ ]:
# Compute ROC curve
# False Positive Rate, True Positive Rate
conf_mtr = confusion_matrix(y_true=y_test,y_pred=y_pred)
# Creating a heatmap for the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_mtr, annot=True, cmap='Blues',
            xticklabels=rf.classes_, yticklabels=rf.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.tight_layout()
plt.show()

In [ ]:
lb = LabelBinarizer()
y_test_bin = lb.fit_transform(y_test)

# Plot ROC curve for each class
plt.figure(figsize=(10, 8))
for i, class_name in enumerate(lb.classes_):
    y_score = rf.predict_proba(x_test)[:, i]
    RocCurveDisplay.from_predictions(
        y_test_bin[:, i],
        y_score,
        name=f'ROC curve for {class_name}',
        ax=plt.gca()
    )

plt.plot([0, 1], [0, 1], linestyle='--', label='No Skill')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()

#### Step-3.2: Applying KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier #Importing KNN
knn = KNeighborsClassifier(n_neighbors= 100) #Changing Value Manually (Due to Performance Reasons)

knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)  # Performing Random Forest

In [ ]:
print(classification_report(y_test, y_pred)) #Printing Classification Report

In [ ]:
cv_knn = cross_val_score(knn, x_train, y_train, cv=5)
print(f'\nCross-validation scores:', ', '.join(map(str, cv_knn)))
print(f'\nMean cross-validation score: {cv_knn.mean():.2f}')

In [ ]:
print(knn.score(x_train, y_train)) #RF Score x_train,y_train

In [ ]:
print(knn.score(x_test, y_test)) #RF Score x_train,y_train

In [ ]:
# Compute Confusion Matrix
# False Positive Rate, True Positive Rate
conf_mtr = confusion_matrix(y_true=y_test, y_pred=y_pred)
# Creating a heatmap for the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_mtr, annot=True, cmap='Blues',
            xticklabels=rf.classes_, yticklabels=rf.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.tight_layout()
plt.show()

In [ ]:
lb = LabelBinarizer()
y_test_bin = lb.fit_transform(y_test)

# Plot ROC curve for each class
plt.figure(figsize=(10, 8))
for i, class_name in enumerate(lb.classes_):
    y_score = rf.predict_proba(x_test)[:, i]
    RocCurveDisplay.from_predictions(
        y_test_bin[:, i],
        y_score,
        name=f'ROC curve for {class_name}',
        ax=plt.gca()
    )

plt.plot([0, 1], [0, 1], linestyle='--', label='No Skill')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()

#### Step-3.3: Applying DNN

In [19]:
model = keras.Sequential()
model.add(layers.Input(shape=(x_train.shape[1],)))  # Input layer

model.add(layers.Dense(128, activation='relu'))  # Hidden layer
model.add(layers.Dropout(0.1))  # Dropout layer

model.add(layers.Dense(256, activation='relu'))  # Hidden layer
model.add(layers.Dropout(0.1))  # Dropout layer

model.add(layers.Dense(128, activation='relu'))  # Hidden layer
model.add(layers.Dropout(0.1))  # Dropout layer

model.add(layers.Dense(6, activation='softmax'))  # Output layer

W0000 00:00:1742299058.073315   38101 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         2,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 69,382 (271.02 KB)

 Trainable params: 69,382 (271.02 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy']) #compiling model


In [22]:
x_val = x_train[:5000]
partial_x_train = x_train[5000:]

y_val = y_train[:5000]
partial_y_train = y_train[5000:]

model.fit(partial_x_train,partial_y_train,epochs=10,batch_size=16,validation_data=(x_val,y_val)) #Fitting Model
model.save("DNN.keras") #Saving the model

Epoch 1/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - accuracy: 0.8815 - loss: 0.4045 - val_accuracy: 0.9674 - val_loss: 0.1111
Epoch 2/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.9625 - loss: 0.1225 - val_accuracy: 0.9728 - val_loss: 0.0905
Epoch 3/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.9693 - loss: 0.0998 - val_accuracy: 0.9778 - val_loss: 0.0772
Epoch 4/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.9719 - loss: 0.0901 - val_accuracy: 0.9790 - val_loss: 0.0702
Epoch 5/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.9742 - loss: 0.0818 - val_accuracy: 0.9790 - val_loss: 0.0733
Epoch 6/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.9765 - loss: 0.0788 - val_accuracy: 0.9812 - val_loss: 0.0682
Epoch 7/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.9776 - loss: 0.0725 - val_accuracy: 0.9822 - val_loss: 0.0631
Epoch 8/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 

In [23]:
print(f"Results of Testing") #Testing the model
model.evaluate(x_test,y_test)

Results of Testing
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 2s 963us/step - accuracy: 0.9804 - loss: 0.0629


[0.060395482927560806, 0.9816266894340515]

In [24]:
print(f"Result of Prediction") #Performing Predictions
predictions=model.predict(x=x_test[:5])
print(predictions)
print("predictions shape:", predictions.shape)

Result of Prediction
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
[[1.0629694e-05 9.5016654e-11 9.9998689e-01 1.3235264e-07 7.7767258e-13
  2.3974942e-06]
 [9.9554092e-01 5.7243386e-13 8.2868297e-08 4.3941643e-03 1.0854998e-06
  6.3672094e-05]
 [1.3040222e-02 9.8684573e-01 4.3854368e-05 1.5235556e-05 7.8715830e-06
  4.7015048e-05]
 [9.9010812e-03 9.9008077e-01 6.0241032e-06 1.0091524e-05 1.9179700e-07
  1.7212184e-06]
 [9.8577207e-03 1.3383859e-12 1.3833181e-05 8.0832488e-06 9.9012029e-01
  1.2329349e-07]]
predictions shape: (5, 6)


#### Step-3.4: Applying DCNN

In [30]:
x_train_reshaped = x_train.reshape(
    x_train.shape[0], x_train.shape[1], 1)

# Now reshape the validation data too:
x_val_reshaped = x_val.reshape(x_val.shape[0], x_val.shape[1], 1)

# Create the model as before
model_cnn = keras.Sequential()

model_cnn.add(layers.Input(shape=(x_train_reshaped.shape[1], 1)))
model_cnn.add(layers.Conv1D(128, 20, data_format='channels_last',
                            activation='relu', padding='same'))
model_cnn.add(layers.Conv1D(256, 20, data_format='channels_last',
                            activation='relu', padding='same'))

model_cnn.add(layers.Flatten())

model_cnn.add(layers.Dense(256, activation='relu',
                           kernel_regularizer=keras.regularizers.l1_l2()))

model_cnn.add(layers.Dense(6, activation='softmax'))

In [31]:
model_cnn.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 20, 128)        │         2,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 20, 256)        │       655,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 5120)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │     1,310,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,970,822 (7.52 MB)

 Trainable params: 1,970,822 (7.52 MB)

 Non-trainable params: 0 (0.00 B)

In [32]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model_cnn.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])  # compiling model

In [33]:
x_val = x_train[:5000]
partial_x_train = x_train[5000:]

y_val = y_train[:5000]
partial_y_train = y_train[5000:]

model_cnn.fit(partial_x_train, partial_y_train, epochs=10, batch_size=16,
              validation_data=(x_val_reshaped, y_val))  # Fitting Model
model_cnn.save("CNN.keras")  # Saving the model

Epoch 1/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 214s 16ms/step - accuracy: 0.9379 - loss: 0.2234 - val_accuracy: 0.9768 - val_loss: 0.0896
Epoch 2/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 208s 15ms/step - accuracy: 0.9765 - loss: 0.0775 - val_accuracy: 0.9800 - val_loss: 0.0685
Epoch 3/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 208s 15ms/step - accuracy: 0.9787 - loss: 0.0679 - val_accuracy: 0.9832 - val_loss: 0.0678
Epoch 4/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 208s 15ms/step - accuracy: 0.9805 - loss: 0.0619 - val_accuracy: 0.9828 - val_loss: 0.0575
Epoch 5/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 208s 15ms/step - accuracy: 0.9818 - loss: 0.0574 - val_accuracy: 0.9830 - val_loss: 0.0545
Epoch 6/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 208s 15ms/step - accuracy: 0.9825 - loss: 0.0552 - val_accuracy: 0.9846 - val_loss: 0.0538
Epoch 7/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 208s 15ms/step - accuracy: 0.9838 - loss: 0.0517 - val_accuracy: 0.9864 - val_loss: 0.0499
Epoch 8/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 208s 15ms/s

In [35]:
print(f"Result of Prediction")  # Performing Predictions
predictions = model_cnn.predict(x=x_test[:5])
print(predictions)
print("predictions shape:", predictions.shape)

Result of Prediction
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
[[3.32424452e-06 1.27538916e-16 9.99996305e-01 4.02626500e-12
  1.09085206e-13 4.01352054e-07]
 [9.99997020e-01 1.37187457e-15 3.95472867e-17 2.94574011e-06
  1.18581941e-11 1.89443058e-12]
 [1.26216337e-01 8.73774111e-01 2.34474598e-07 2.75531988e-06
  6.67841641e-06 5.19055332e-09]
 [1.36107700e-02 9.86311257e-01 5.87790673e-05 6.19983075e-06
  3.45321922e-07 1.25505940e-05]
 [6.43145561e-01 9.48597574e-18 2.88383653e-05 2.30473375e-07
  3.56825352e-01 1.02781828e-11]]
predictions shape: (5, 6)
